# BPE Tokenizer

Pada kode ini, ditunjukkan contoh penggunaan BPE Tokenizer buatan HuggingFace. [Petunjuk instalasi package](https://huggingface.co/docs/tokenizers/installation)

In [ ]:
# Import semua packages yang diperlukan.
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [ ]:
# Instantiate tokenizer BPE dengan unknown token [UNK] (berguna 
# saat testing jika menemukan misalnya karakter yang tidak dikenali).
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [ ]:
# Instantiate trainer BPE, PERHATIKAN hyperparameter vocab_size yang konsepnya
# mirip dengan `num_of_merges` yang diajarkan di kelas (relasinya kurang lebih 
# num_of_merges = vocab_size - num_of_alphabet).
trainer = BpeTrainer(
    vocab_size=100,  # Perbesar/perkecil dan lihat bedanya pada hasil tokenisasi
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
)

In [ ]:
# Menambahkan pre-tokenizer yang berfungsi untuk memecah tiap kalimat/dokumen
# menjadi individual words berdasarkan keberadaan whitespace.
tokenizer.pre_tokenizer = Whitespace()

In [ ]:
# Mengekstrak kalimat-kalimat korpus ke dataset tersendiri sesuai spesifikasi
# input dari package tokenizer, yaitu file berisi raw text (tanpa hasil tokenisasi)
with open('train.txt', 'r') as train, open('train_text_only.txt', 'w') as out:
    lines = train.readlines()
    for line in lines:
        if line.startswith('# text ='):
            out.write(line[9:])

In [ ]:
# Melatih tokenizer dengan corpus.
tokenizer.train(
    files=["train_text_only.txt"],
    trainer=trainer
)

In [ ]:
# Contoh melakukan tokenisasi suatu kalimat
print(tokenizer.encode('Perusahaan pertambangan minyak').tokens)